# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas

import pandas as pd
import numpy as np
from pandas import RangeIndex

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset.

In [2]:
# Run this code:

admissions = pd.read_csv('../Admission_Predict.csv')
#to remove trailing spaces at the end of the column names
admissions.columns = admissions.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [3]:
# Your code here:
admissions.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [4]:
# Your code here:
admissions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
Serial No.           385 non-null int64
GRE Score            385 non-null int64
TOEFL Score          385 non-null int64
University Rating    385 non-null int64
SOP                  385 non-null float64
LOR                  385 non-null float64
CGPA                 385 non-null float64
Research             385 non-null int64
Chance of Admit      385 non-null float64
dtypes: float64(4), int64(5)
memory usage: 27.1 KB


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [5]:
# Your code here:
admissions.set_index('Serial No.', inplace=True)
admissions.head()


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [6]:
# Your code here:

admissions.drop_duplicates(subset=['GRE Score', 'TOEFL Score'], inplace=True)

admissions.set_index(['GRE Score','TOEFL Score'], inplace=True)
admissions.head()


,,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
GRE Score,TOEFL Score,,,,,,
337,118,4,4.5,4.5,9.65,1,0.92
316,104,3,3.0,3.5,8.00,1,0.72
322,110,3,3.5,2.5,8.67,1,0.80
314,103,2,2.0,3.0,8.21,0,0.65
330,115,5,4.5,3.0,9.34,1,0.90


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [7]:
# Your code here:
admissions.reset_index(inplace=True)


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [8]:
sum(admissions.eval('CGPA > 9 & Research == 1'))

68

In [9]:
# Your code here:

admissions.loc(admissions.CGPA > 9 & admissions.Research == 1).all()


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
sum(admissions.eval('CGPA > 9 & Research == 1'))


68

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# Your code here:

np.mean(admissions.query('CGPA > 9 & SOP < 3.5')['Chance of Admit'])


0.7975

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    # Your code here:
    return (col - np.mean(col)) / np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [13]:
# Your code here:
admissions['CGPA_std'] = standardize(admissions.CGPA)
admissions['GRE_std'] = standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions.LOR)


We will generate the decision choice at random using the code below. Please run the cell.

In [14]:
# Run this code:

from random import choices
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [15]:
# Your code here:
decision_choice


['CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'C

In [16]:
admissions['deciding_column'] =admissions.lookup(admissions.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [17]:
# Your code here:

admissions['decision'] = np.where(admissions.deciding_column > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [18]:
# Your code here:

admissions.decision.value_counts()

0    206
1     66
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [19]:
admissions.columns = (admissions.columns
                            .str.replace(' ', '_')
                            .str.replace('.', '')
                            .str.lower())


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [21]:
admissions['adjusted_gre'] = pd.cut(np.where(admissions.university_rating >=4, admissions.gre_score + 10, admissions.gre_score), bins=4)


In [22]:
admissions.head()


,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,118,4,4.5,4.5,9.65,1,0.92,1.728179,1.736205,1.151800,1.728179,1,"(335.0, 350.0]"
1,316,104,3,3.0,3.5,8.00,1,0.72,-0.902066,-0.004571,0.075203,-0.004571,0,"(305.0, 320.0]"
2,322,110,3,3.5,2.5,8.67,1,0.80,0.165973,0.492793,-1.001393,0.165973,0,"(320.0, 335.0]"
3,314,103,2,2.0,3.0,8.21,0,0.65,-0.567308,-0.170360,-0.463095,-0.463095,0,"(305.0, 320.0]"
4,330,115,5,4.5,3.0,9.34,1,0.90,1.234011,1.155946,-0.463095,1.155946,1,"(335.0, 350.0]"
